### Import Python packages 

In [8]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import decimal
from IPython.display import HTML

### Creating list of filepaths to process original event csv data files

In [9]:
# checking your current working directory
#print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(*file_path_list, sep="\n")

/workspace/home/event_data/2018-11-20-events.csv
/workspace/home/event_data/2018-11-12-events.csv
/workspace/home/event_data/2018-11-04-events.csv
/workspace/home/event_data/2018-11-10-events.csv
/workspace/home/event_data/2018-11-06-events.csv
/workspace/home/event_data/2018-11-09-events.csv
/workspace/home/event_data/2018-11-05-events.csv
/workspace/home/event_data/2018-11-17-events.csv
/workspace/home/event_data/2018-11-30-events.csv
/workspace/home/event_data/2018-11-29-events.csv
/workspace/home/event_data/2018-11-16-events.csv
/workspace/home/event_data/2018-11-26-events.csv
/workspace/home/event_data/2018-11-22-events.csv
/workspace/home/event_data/2018-11-13-events.csv
/workspace/home/event_data/2018-11-25-events.csv
/workspace/home/event_data/2018-11-21-events.csv
/workspace/home/event_data/2018-11-11-events.csv
/workspace/home/event_data/2018-11-02-events.csv
/workspace/home/event_data/2018-11-14-events.csv
/workspace/home/event_data/2018-11-03-events.csv
/workspace/home/even

### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [10]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [11]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


### Creating a Cluster

In [12]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

### Create Keyspace

In [13]:
# Create a Keyspace T
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS t 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

### Set Keyspace

In [14]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('t')
except Exception as e:
    print(e)


#### -----------------------------------------------------------------

### CODE FOR QUESTION 1 FOR THE DATA: 
#### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
COMPOSITE KEY:
Using column "iteminsession" as the partition key will distribute data items equaly in nodes for small amount of session items. 
Using clustering column "sessionid" will allow us to use it in the WHERE clause and insure the primary key is unique


##### Table creation and data loading

In [15]:
query1 = "CREATE TABLE IF NOT EXISTS song_length_sessionitem " 
query1 = query1 + "(iteminsession int, sessionid int, artist text, song text, length decimal, PRIMARY KEY(iteminsession, sessionid))"
try:
    session.execute(query1)
except Exception as e:
    print(e)       

In [16]:
file = 'event_datafile_new.csv'
##INSERT statements
query = "INSERT INTO song_length_sessionitem (iteminsession, sessionid, artist, song, length)"  
query = query + " VALUES (%s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Column element assigned for each column in the INSERT statement.
        ## see above insert statements for column name mapping  
        session.execute(query, (int(line[3]), int(line[8]), line[0], line[9], decimal.Decimal(line[5])))

##### Do a SELECT to verify that the data have been inserted into table

In [17]:
# SELECT statement to verify the data
query = "SELECT artist, song, length FROM song_length_sessionitem WHERE iteminsession = 4 and sessionid=338"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df = pd.DataFrame(list(rows))
HTML(df.to_html(index=False))

artist,song,length
Faithless,Music Matters (Mark Knight Dub),495.3073


#### -----------------------------------------------------------------

### CODE FOR QUESTION 2 FOR THE DATA: 
#### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
COMPOSITE KEY:
Partition is by "userid, sessionid" on nodes for our small amount of data and clustering column "itemsession" is used to display in ascending order, so we are satisfying the "itemsession" ordering request

##### Table creation and data loading

In [18]:
query2 = "CREATE TABLE IF NOT EXISTS song_playlist_session " 
query2 = query2 + "(userid int, sessionid int, iteminsession int, artist text, song text, user text, PRIMARY KEY((userid,sessionid), iteminsession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)       

                    

In [19]:
file = 'event_datafile_new.csv'
##INSERT statements
query = "INSERT INTO song_playlist_session (userid, sessionid, iteminsession, artist, song, user)"  
query = query + " VALUES (%s, %s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Column element assigned for each column in the INSERT statement.
        ##see above insert statements for column name mapping 
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], (line[1]+' '+ line[4])))

##### Do a SELECT to verify that the data have been inserted into table

In [20]:
# SELECT statement to verify the data
query = "SELECT artist, song, user FROM song_playlist_session WHERE userid=10 and sessionid=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame(list(rows))
HTML(df.to_html(index=False))

artist,song,user
Down To The Bone,Keep On Keepin' On,Sylvie Cruz
Three Drives,Greece 2000,Sylvie Cruz
Sebastien Tellier,Kilometer,Sylvie Cruz
Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


#### -----------------------------------------------------------------

### CODE FOR QUESTION 3 FOR THE DATA: 
#### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
COMPOSITE KEY:
Partition is by "song" on nodes for our small amount of data and clustering column "userid" insure a unique primary key for the specific question 3 

##### Table creation and data loading

In [21]:
query3 = "CREATE TABLE IF NOT EXISTS song_listened_by_user " 
query3 = query3 + "(song text, userid int, user text, PRIMARY KEY(song, userid))"
try:
    session.execute(query3)
except Exception as e:
    print(e)       

                    

In [22]:
file = 'event_datafile_new.csv'
##INSERT statements
query = "INSERT INTO song_listened_by_user (song, userid, user)"  
query = query + " VALUES (%s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Column element assigned for each column in the INSERT statement.
        ##see above insert statements for column name mapping 
        session.execute(query, (line[9], int(line[10]), (line[1]+' '+ line[4])))

##### Do a SELECT to verify that the data have been inserted into table

In [23]:
# SELECT statement to verify the data
query = "SELECT user FROM song_listened_by_user WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame(list(rows))
HTML(df.to_html(index=False))

user
Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [24]:
## Dropping the table before closing out the sessions
try:
   session.execute('DROP TABLE song_length_sessionitem')
   session.execute('DROP TABLE song_playlist_session')
   session.execute('DROP TABLE song_listened_by_user')
except Exception as e:
    print(e)       

### Close the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()